In [ ]:
from google.colab import drive
drive.mount('/gdrive/')
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive/
Mounted at /content/gdrive/


In [ ]:
import sys
sys.path.append('/content/gdrive/My Drive/Colab Notebooks/SacnnKeras/DataPreparation')
sys.path.append('/gdrive/My Drive/Colab Notebooks/SacnnKeras')

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
a=cv2.imread('/content/gdrive/My Drive/Colab Notebooks/SacnnKeras/datasets/ShanghaiTech/train_data/images/1.jpg')
#cv2_imshow(a)

In [ ]:
from keras.layers import Conv2D,Activation,MaxPool2D,Dense,Conv2DTranspose,Concatenate,Input,Flatten;
from tensorflow.keras.applications import VGG19
from keras.models import Sequential,Model
import keras

Using TensorFlow backend.


In [ ]:
img_width, img_height = 500, 500
train_data_dir = "/content/gdrive/My Drive/Colab Notebooks/SacnnKeras/datasets/ShanghaiTech/formatted_trainval/Shanghaitech_patches_9/train"
validation_data_dir = "/content/gdrive/My Drive/Colab Notebooks/SacnnKeras/datasets/ShanghaiTech/formatted_trainval/Shanghaitech_patches_9/val"
nb_train_samples = 700
nb_validation_samples = 498
batch_size = 1
epochs = 250

In [ ]:
# Model design
img_input=Input(shape=(img_width,img_height,3))

# Block 1
x11 = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
x12 = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x11)
xp1 = MaxPool2D((2, 2), strides=(2, 2), name='block1_pool')(x12)

# Block 2
x21 = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(xp1)
x22 = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x21)
xp2 = MaxPool2D((2, 2), strides=(2, 2), name='block2_pool')(x22)

# Block 3
x31 = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(xp2)
x32 = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x31)
x33 = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x32)
x34 = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv4')(x33)
xp3 = MaxPool2D((2, 2), strides=(2, 2), name='block3_pool')(x34)

# Block 4
x41 = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(xp3)
x42 = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x41)
x43 = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x42)

#POOL4
xp4=MaxPool2D(pool_size=(2, 2), strides=2)(x43)       

#Block5
x51=Conv2D(512,(3,3),activation='relu', padding='same', name='sacnn5_conv1')(xp4)
x52=Conv2D(512,(3,3),activation='relu', padding='same', name='sacnn5_conv2')(x51)
x53=Conv2D(512,(3,3),activation='relu', padding='same', name='sacnn5_conv3')(x52)

#POOL5
xp5=MaxPool2D(pool_size=(3, 3), strides=1,padding='same')(x53)

#CONV6_1
x61=Conv2D(512,(3,3),activation='relu', padding='same', name='sacnn6_conv1')(xp5)

#MERGE 6_1 AND 5_3
x6153=Concatenate(axis=3)([x53,x61])

#DECONVOLUTION OR UPSAMPLE
xd=Conv2DTranspose(512, (2,2), strides=(1, 1), activation='relu',padding='same')(x6153)
print(xd)

#MERGE DECONV + 4_3
xd43=Concatenate(axis=3)([xd,xp4])

#BLOCK7
x71=Conv2D(512,(3,3),activation='relu', padding='same', name='sacnn7_conv1')(xd43)
x72=Conv2D(256,(3,3),activation='relu', padding='same', name='sacnn7_conv2')(x71)
x73=Conv2D(1,(1,1),activation='relu', padding='same', name='sacnn7_conv3')(x72)

sacnn=Model(inputs=img_input, outputs=x73)





Tensor("conv2d_transpose_1/Relu:0", shape=(?, ?, ?, 512), dtype=float32)


In [ ]:
sacnn.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 500, 500, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 500, 500, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 500, 500, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 250, 250, 64) 0           block1_conv2[0][0]               
____________________________________________________________________________________________

In [ ]:
#custom metrics 
def mean_square(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

def mean_absolute(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=2)

# Compile the model
sacnn.compile(optimizer=SGD(lr=1e-6,momentum=0.9), loss='mean_squared_error', metrics=['accuracy',mean_square,mean_absolute])

model.summary()
